In [2]:
import numpy as np
import csv
from scipy.optimize import minimize, least_squares

### Baby Boomer Divorce Rates

The file babyboomerdivorce.csv contains data on the rate of baby-boomer divorce. The problem has two independent variables: the marriage age and years of school, denoted by x and y respectively.

a. Formulate a linear regression learning problem for each of the following buckets of models.

$\\$ Model 1 : $A = \begin{bmatrix} 
	1 & x_1 & y_1 \\
	1 & x_2 & y_2 \\
    .. & .. & .. \\
	1 & x_n & y_n \\
	\end{bmatrix}$, $\theta = [\theta_{00}, \theta_{10}, \theta_{01}]^T$

$argmin_{\theta}||A\theta - b||_2$

$\\$ Model 2 : $A = \begin{bmatrix} 
	1 & x_1 & y_1 & x_1y_1 \\
	1 & x_2 & y_2 & x_2y_2 \\
    .. & .. & .. \\
	1 & x_n & y_n & x_ny_n \\
	\end{bmatrix}$ $\theta = [\theta_{00}, \theta_{10}, \theta_{01}, \theta_{11}]^T$

$argmin_{\theta}||A\theta - b||_2$

$\\$ Model 3 : $A = \begin{bmatrix} 
	1 & x_1 & y_1 & x_1y_1 & x_1^2 & y_1^2\\
	1 & x_2 & y_2 & x_2y_2 & x_2^2 & y_2^2\\
    .. & .. & .. \\
	1 & x_n & y_n & x_ny_n & x_n^2 & y_n^2 \\
	\end{bmatrix}$, $\theta = [\theta_{00}, \theta_{10}, \theta_{01}, \theta_{11}, \theta_{20}, \theta_{02}]^T$

$argmin_{\theta}||A\theta - b||_2$

b. Solve the learning problems that you formulated in part (a) and report he best fit parameter values and uncertainties in each parameter.

$\\$ The best fit parameters were within the third model (the full results posted in the code below). $\theta = $[ 1.14478739e+02 -2.17977168e+00 -3.05970381e-02  1.00666766e-01
 -1.87273229e-02 -1.67386364e-01]. Right away it is troubling that the first theta value, $\theta_{00}$, is so much larger than the other parameters.

c. Calculate and report the leave-one-out cross validation error for each bucket of models in part (a). Which bucket performs best according to this metric?

$\\$ However calculating the leave-one-out cross validation error the first model, Model 1 with only 3 parameters, had the best Cost Value.

d. Formulate a linear regression problem for the bucket of models using the 1-norm. Solve the problem using a convex optimization solver and report the best fit parameter values.

$\\$ We can use the same formulations for the models as posted above, however now with $theta^* = argmin_{\theta}||A\theta - b||_1$. Then we can use scipy to calculate the theta values that obtain a minimum. The best fit parameters for Model 1 were $\theta = $[126.73699162  -1.84671145  -2.30199812].

In [3]:
def get_data():
    with open("f1.csv") as f:
        c = csv.reader(f)
        x = []
        y = []
        b = []
        for row in c:
            try:
                x.append(float(row[0]))
                y.append(float(row[1]))
                b.append(float(row[2]))
            except Exception:
                continue
    return x,y,b

x,y,b = get_data()
b = np.array(b)

m = len(x)

def A_matrix(x,y,m, model=1):
    if model==1:
        return np.array([[1, x[i], y[i]] for i in range(m)])
    if model==2:
        return np.array([[1, x[i], y[i], x[i]*y[i]] for i in range(m)])
    if model==3:
        return np.array([[1, x[i], y[i], x[i]*y[i], x[i]**2,  y[i]**2] for i in range(m)])


parameters = []

for model_num in range(1,4):
    A = A_matrix(x,y,m, model=model_num)
    c = np.linalg.lstsq(A,b,rcond=None)
    error = np.linalg.norm(b-A@c[0], ord=2)
    print("Model " + str(model_num) + ". Cost: " + str(error) + " ---- Theta: " + str(c[0]))
    parameters.append(c[0])


Model 1. Cost: 19.679551485957365 ---- Theta: [122.18308858  -1.88084719  -1.94914286]
Model 2. Cost: 15.836753427794445 ---- Theta: [ 1.67870700e+02 -3.34051530e+00 -5.10001262e+00  1.00666766e-01]
Model 3. Cost: 13.855672523516391 ---- Theta: [ 1.14478739e+02 -2.17977168e+00 -3.05970381e-02  1.00666766e-01
 -1.87273229e-02 -1.67386364e-01]


In [4]:
errors = []

# for set of parameters learned from the bucket of models
for model_num in range(1,4):
    A = A_matrix(x,y,m, model=model_num)
    err = 0
    for j in range(m):
        A_new = np.copy(A)
        b_new = np.copy(b)
        for k in range(len(A[0])):
            A_new[j][k] = 0
        b_new[j] = 0
        c = np.linalg.lstsq(A,b_new,rcond=None)[0]
        err += (b[j] - (A[j]@c))**2
    errors.append(err/m)

for model_num in range(1,4):
    print("Model " + str(model_num) + ". Error: " + str(errors[model_num-1]))

Model 1. Error: 58.27586600227191
Model 2. Error: 97.56606357524058
Model 3. Error: 202.1598284890386


In [5]:
def get_data():
    with open("f1.csv") as f:
        c = csv.reader(f)
        x = []
        y = []
        b = []
        for row in c:
            try:
                x.append(float(row[0]))
                y.append(float(row[1]))
                b.append(float(row[2]))
            except Exception:
                continue
    return x,y,b

x,y,b = get_data()
b = np.array(b)
m = len(x)

def cost(c):
    out = A_matrix(x,y,m)@c - b
    return np.linalg.norm(out, ord=1)

v = minimize(cost, [1,1,1])
theta = v["x"]
print("Cost: " + str(cost(theta)) + " ---- Theta: " + str(theta))

Cost: 66.42747385921943 ---- Theta: [126.73699162  -1.84671145  -2.30199812]


### 2. Analysis of an Enzyme Reaction

The file salmonella.csv contains data for the activity of a colony of salmonella as a function of temperature (in
Celsius). These data should be fit to the following model:

$$f_{\theta}(T) = \frac{S_Be^{-E_B/T}}{S_Ae^{-E_A/T} + S_Be^{-E_B/T} + S_Ce^{-E_C/T}}$$ 

a. Show that this bucket of models has two structurally unidentifiable parameters by rewriting it in terms of the following identifiable combinations:

First we can simplify this model by eliminating two parameters. We can do this by multiplying the top and bottom of the equation by $S_A e^{E_A/T}$ eliminating one term with parameters from the denominator. This gives us:

$$f_{\theta}(T) = \frac{S_Ae^{E_A/T}}{S_Ae^{E_A/T}}\frac{S_Be^{-[E_B-E_A]/T}}{1 + \frac{S_B}{S_A}e^{-[E_B-E_A]/T} + \frac{S_C}{S_A}e^{-[E_C-E_A]/T}}$$ 

Thus now instead of our parameters being $\theta = (S_A, E_A, S_B, E_B, S_C, E_C)'$ our parameters are now $\theta = (\frac{S_B}{S_A}, E_B-E_A,\frac{S_C}{S_A}, E_C-E_A)'$. Meaning that we had two structurally unidentifiable parameters. (Theta now only consists of 4 parameters rather than 6)

b. Fit the 4-parameter model that you found in part (a) and plot the fit against the data. This model can be challenging to fit, so use the visualizations in part (b) to guide your initial parameter guess. (Hint 1: Be sure to convert your temperatures to Kelvin. Hint 2: Each of the parameters in part (a) should be positive. I found it easier to find a good fit when using the log of the parameter values.

#### I fit the model using the 1, 2, and inf norms. The results (as printed in the code box below were):

1 Norm. Cost: 3.235645430375456 ---- $\theta = $ [ 5.59630525e-03  4.90431008e+01 -3.63701102e-01 -3.18694045e+00]

2 Norm. Cost: 0.41037943460984144 ---- $\theta = $[ 4.94778972e-06 -2.93127612e+02  1.48224929e-04 -3.75514224e+02]

Inf Norm. Cost: 0.32866120134337834 ----$\theta = $[4.00099523e-03 6.29808162e+00 1.06749999e+00 6.36034742e+00]

####  Additionally using the hint given that the parameters should all be positive, the results were:

1 Norm. Cost: 7.6183763820743104 ---- $\theta = $ [0.00611732 1.2085457  2.57618796 0.92051712]

2 Norm. Cost: 0.8328369308500786 ---- $\theta = $ [  0.87965523  31.10739387 174.9768124    0.        ]

Inf Norm. Cost: 0.3748165141454243 ---- $\theta = $ [0.00602286 1.11448707 2.70400066 0.92174118]

c. Calculate the Fisher Information Matrix for your model at the best fit and report your estimated uncertainty in each parameter. Be sure to include the factor of σ in the FIM.

The Fisher Information Matrix is found by multiplying the Jacobian matrix by the Jacobian Transpose. The Jacobian was calulated by finding the derivative relative to each term $\theta_i$ represtend in the ith column of the matrix and in each row evaluating the derivatives for each value T.

( 2.58502709e+00  9.38464382e-04 -4.16096087e-03  1.03707321e-02

 9.38464382e-04  4.48282270e-07 -1.50571622e-06  3.75283018e-06

 -4.16096087e-03 -1.50571622e-06  6.69786632e-06 -1.66936867e-05

 1.03707321e-02  3.75283018e-06 -1.66936867e-05  4.16071571e-05 )

 Next, to estimate parametric uncertainty we can find the product of $\sigma^2 * (A'A)^{-1} = FIM(\theta)$. In general the parametric uncertainty appears to be low.


In [30]:
# get the data
def get_data():
    with open("f2.csv") as f:
        c = csv.reader(f)
        x = []
        b = []
        for row in c:
            try:
                x.append(float(row[0]))
                b.append(float(row[1]))
            except Exception:
                continue
    return x,b

# read and formalize as a numpy array
T_vals,b = get_data()
b = np.array(b)


# uses the formula derived from part b. Multiplies by 255.927778* to convert from Fahrenheit to Kelvin
def f(theta):
    x = [255.927778*theta[0]*np.exp(-theta[1]/T)/(1 + theta[0]*np.exp(-theta[1]/T) + theta[2]*np.exp(-theta[3]/T)) for T in T_vals]
    return np.linalg.norm([x[i] - b[i] for i in range(len(b))], ord=2)

def g(theta):
    x = [255.927778*theta[0]*np.exp(-theta[1]/T)/(1 + theta[0]*np.exp(-theta[1]/T) + theta[2]*np.exp(-theta[3]/T)) for T in T_vals]
    return np.linalg.norm([x[i] - b[i] for i in range(len(b))], ord=1)

def h(theta):
    x = [255.927778*theta[0]*np.exp(-theta[1]/T)/(1 + theta[0]*np.exp(-theta[1]/T) + theta[2]*np.exp(-theta[3]/T)) for T in T_vals]
    return np.linalg.norm([x[i] - b[i] for i in range(len(b))], ord=np.Inf)


print("Without Bounds: ")
theta = minimize(f,[1,1,1,1])
print("2 Norm. Cost: " + str(f(theta["x"])) + " ---- Theta: " + str(theta["x"]))

theta = minimize(g,[1,1,1,1])
print("1 Norm. Cost: " + str(g(theta["x"])) + " ---- Theta: " + str(theta["x"]))

theta = minimize(h,[1,1,1,1])
print("Inf Norm. Cost: " + str(h(theta["x"])) + " ---- Theta: " + str(theta["x"]))

print()
print("With Bounds: ")

bnds = ((0, None), (0, None), (0, None), (0, None))
theta = minimize(f,[1,1,1,1], bounds=bnds)
print("2 Norm. Cost: " + str(f(theta["x"])) + " ---- Theta: " + str(theta["x"]))

theta = minimize(g,[1,1,1,1], bounds=bnds)
print("1 Norm. Cost: " + str(g(theta["x"])) + " ---- Theta: " + str(theta["x"]))

theta = minimize(h,[1,1,1,1], bounds=bnds)
print("Inf Norm. Cost: " + str(h(theta["x"])) + " ---- Theta: " + str(theta["x"]))

Without Bounds: 
2 Norm. Cost: 0.41037943460984144 ---- Theta: [ 4.94778972e-06 -2.93127612e+02  1.48224929e-04 -3.75514224e+02]
1 Norm. Cost: 3.235645430375456 ---- Theta: [ 5.59630525e-03  4.90431008e+01 -3.63701102e-01 -3.18694045e+00]
Inf Norm. Cost: 0.32866120134337834 ---- Theta: [4.00099523e-03 6.29808162e+00 1.06749999e+00 6.36034742e+00]

With Bounds: 
2 Norm. Cost: 0.8328369308500786 ---- Theta: [  0.87965523  31.10739387 174.9768124    0.        ]
1 Norm. Cost: 7.6183763820743104 ---- Theta: [0.00611732 1.2085457  2.57618796 0.92051712]
Inf Norm. Cost: 0.3748165141454243 ---- Theta: [0.00602286 1.11448707 2.70400066 0.92174118]


In [28]:
# 2 Part C - Calculate the FIM and report the estimated uncertainty in each parameter.
# The Fisher Information Matrix is found by multiplying the Jacobian by the Jacobian transpose

# Jacobian Matrix
def jacob(theta):
    J = np.zeros((len(b),4))

    for i in range(len(b)):
        T = T_vals[i]
        Top = theta[0]*np.exp(-theta[1]/T)
        Bottom = (1 + theta[0]*np.exp(-theta[1]/T) + theta[2]*np.exp(-theta[3]/T))

        # the derivatives with respect to theta_1, theta_2, theta_3, theta_4 evalutated at each T value T_i (so each function derivative below is in column i)
        J[i][0] = (np.exp(-theta[1]/T)*Bottom - theta[0]*np.exp(-2*theta[1]/T)) / Bottom**2
        J[i][1] = (theta[1]/T*theta[0]*np.exp(-theta[1]/T)*Bottom - theta[1]**2*-theta[1]/T*Top ) / Bottom**2
        J[i][2] = -1*Top/Bottom**2*(np.exp(-theta[3]/T))
        J[i][3] = Top/Bottom**2*theta[2]*theta[3]*(np.exp(-theta[3]/T))

    return J

# Fisher Information Matrix
def FIM(theta):
    j = jacob(theta)
    return j.T @ j

print(FIM([0.00602286, 1.11448707, 2.70400066, 0.92174118]))

[[ 2.58502709e+00  9.38464382e-04 -4.16096087e-03  1.03707321e-02]
 [ 9.38464382e-04  4.48282270e-07 -1.50571622e-06  3.75283018e-06]
 [-4.16096087e-03 -1.50571622e-06  6.69786632e-06 -1.66936867e-05]
 [ 1.03707321e-02  3.75283018e-06 -1.66936867e-05  4.16071571e-05]]


### 3. Two-Norm Penalty for Weights

Derive a close-form expression for the solution to the following variation on the typical least-squares problem. where A, b and $\lambda$ are given, but no rank or dimension assumptions on A can be guaranteed. Hint: Try to reduce this problem to the standard least squares problem for a new matrix $\hat{A}$ and a vector $\hat{v}$ and then apply a solution to this problem.

We can find the closed form solution by finding an explicit formula for $(||Ax-b||_2^2 + \lambda ||x||_2^2)$, taking the derivative and setting the expression equal to 0. Let, 

$$f(x) = (||Ax-b||_2^2 + \lambda ||x||_2^2)$$

First from HW 2 we know that $$||Ax-b||_2^2 = \sum_i^m [ \sum_j^n \sum_k^n A_{ij}x_jA_{ik}x_k-2b_i \sum_j^n (A_{ij}x_j)+b_i^2]$$

Additionally $\lambda ||x||_2^2 = \lambda \sum_i^n x_i^2$. Thus combinding the two expressions we have,

$$f(x) = \sum_i^m [ \sum_j^n \sum_k^n A_{ij}x_jA_{ik}x_k-2b_i \sum_j^n (A_{ij}x_j)+b_i^2] + \lambda \sum_i^n x_i^2$$

Then taking the derivative of the function we have:

$$\frac{df}{dx_{\mu}} = 2\sum_i^n (A_{i \mu} \sum A_{ij}x_j - b_iA_{i\mu}) + 2 \lambda x_{\mu}$$

Which we can rewrite as:

$$\frac{df}{dx} = 2[A'Ax - A'b + \lambda x]$$

Then in order to find the minimum value we set the derivative $\frac{df}{dx} = 2[A'Ax - A'b + \lambda x] = 0$. 

$$\frac{df}{dx} = 2[A'Ax - A'b + \lambda x] = 0$$

$$ \implies A'Ax - A'b + \lambda x = 0$$

$$ \implies A'Ax + \lambda x = A'b$$

$$ \implies x(A'A + \lambda) = A'b$$

$$ \implies x^* = (A'A + \lambda)^{-1}A'b$$

Lastly if desired we can check that this is a minimum by checking that the second derivative is positive. However this follows directly from previous work from HW 2.

## Part 2

### 4. Overdetermined Linear Regression is Biased

Consider a linear least squares regression model. Assume the errors are normally distributed according to e ~ $N(0, \sigma^2 I)$ and that there are more parameters than data points so that A is a fat matrix. Show that the estimator $\hat{\theta} = A^+ y$ is biased. 

First Note that the pseudo-inverse $A^+$ is defined by $A^+ = (A'A)^{-1}A'$. Additionally we know that the bias of the esimator is defined by $bias(\hat{\theta}) = <\hat{\theta}> - \theta$.

Then, 

$$bias(\theta, \hat{\theta}) = <\hat{\theta}> - \theta$$
$$ = <A^+y> - \theta$$
$$ = <A^+(A\theta + e)> - \theta$$
$$ = <A^+A\theta + A^+e> - \theta$$

By the definition of pseudo-inverse $A^+A = I$.


$$ \implies  bias(\theta, \hat{\theta}) = <\theta + A^+e> - \theta$$

The expectation <> is a linear operator, thus we can distribute through addition. Thus this becomes,

$$ = <\theta> + <A^+e> - \theta$$
$$ = \theta + <A^+e> - \theta$$
$$ = <A^+e> \neq 0$$

Thus $bias(\theta, \hat{\theta}) \neq 0$ and so by definition the model is biased.

$\\$

### 5 Poisson Distribution from the Binomial Distribution

The binomial distribution is a two-parameter family of distributions given by:

$$ P(m;n,p) = (n \ m)p^m(1-p)^{n-m}$$

If you have a coin that has probability p of coming up heads and you flip it n times, then P gives the probability of getting m heads. In this two-parameter bucket of models the parameters satisify $p \in [0,]$ and $n \in N$.


Models of probability distributions, such as this, have regimes of practically unidentificable parameters just as nonlinear regression models do. Show that in the regime $p \to 0$ and $n \to \infty$, the parameter $\lambda = np$ is the identifiable combination. Whas is an expression for the probability distribution in this regime? (Hint: Define $\lambda = np$, replace $p=\lambda/n$ and take the limit $n \to \infty$.)

Admittedly from the title of the problem, I was first inspired to try to prove that this Distribution will become a Poisson Distribution. The formula for a Poisson Distribution is given by,

$$Pois(\lambda, m) =  \frac{\lambda^m e^{-m}}{m!}$$

First we will note that the choose function $(n \ m) = \frac{n!}{m!(n-m)!}$.

As recommended in the hint, we will set $\lambda = np$ and $p = \lambda / n$ so that we can more simply just take the limit as $n \to \infty$.

Then, 

$$lim_{n \to \infty} P(m;n,p) =  lim_{n \to \infty} P(m;n,\lambda/n)$$ 

$$ = lim_{n \to \infty} \frac{n!}{m!(n-m)!} (\frac{\lambda}{n})^m (1-\frac{\lambda}{n})^{n-m}$$

In order to match the Poisson Distribution, we will first take the $\frac{1}{m!}$ and $\lambda^m$ terms out to the outside of the limit,

$$ =\frac{\lambda^m}{m!} lim_{n \to \infty} \frac{n!}{(n-m)!} (\frac{1}{n})^m (1-\frac{\lambda}{n})^{n-m}$$

Next we can distribute the limit operator such that,

$$ \frac{\lambda^m}{m!} lim_{n \to \infty} \frac{n!}{(n-m)!} (\frac{1}{n})^m (1-\frac{\lambda}{n})^{n-m} = \frac{\lambda^m}{m!} lim_{n \to \infty} [\frac{n!}{(n-m)!} (\frac{1}{n})^m ] lim_{n \to \infty} [(1-\frac{\lambda}{n})^{n-m}]$$

$$ = \frac{\lambda^m}{m!} lim_{n \to \infty} (1-\frac{\lambda}{n})^{n-m}$$

Since $lim_{n \to \infty} \frac{n!}{(n-m)!} (\frac{\lambda}{n})^m = lim_{n \to \infty} \frac{\lambda^m n!}{n(n-m)!} = lim_{n \to \infty} \frac{\lambda^m (n-1)!}{(n-m)!} = \lambda^m$.

Lastly the limit,

$$lim_{n \to \infty} (1-\frac{\lambda}{n})^{n-m} = lim_{n \to \infty} (1-\frac{\lambda}{n})^{n} \ lim_{n \to \infty} (1-\frac{\lambda}{n})^{-m}$$

$$= lim_{n \to \infty} (1+\frac{1}{n})^{-\lambda n} \ lim_{n \to \infty} (1+\frac{1}{n})^{\lambda m}$$

$$= e^{-\lambda} \ (1)^{\lambda m} = e^{-\lambda}$$

Thus when put all together we have,

$$ lim_{n \to \infty} P(m;n,p) = \frac{\lambda^m e^{-\lambda}}{m!} = Pois(\lambda, m)$$

Thus the Binomial Distribution becomes a Poisson Distribution with the regime $p \to 0$ and $n \to \infty$.






